## 前期准备preparation in advance

In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import random
import time
import pandas
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Set random seeds to ensure reproducibility
# 设置随机种子以保证可复现
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [3]:
#加载数据
def load_data(folder_path):
    # Retrieve all CSV files from the folder
    # 获取文件夹中的所有 CSV 文件
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]  
    # List used to store data
    # 用来存放数据的列表
    data_list = []  
    
    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        # Read CSV file
        # 读取 CSV 文件
        df = pd.read_csv(file_path,encoding='gbk') 
        #Only the first and tenth columns of data (including label data) have been selected here and can be freely adjusted
        #这里只选中了第1列与第10列数据（包括标签数据），可自由调整
        select_data=df.iloc[:,1:10]
        #Assuming that the number of rows in each file is seq_1en and the number of columns is feature_stize
        # 假设每个文件的行数是 seq_len，列数是 feature_size
        seq_len = select_data.shape[0]  
        feature_size = select_data.shape[1]  
        #Convert DataFrame to NumPy array
        # 将 DataFrame 转换为 NumPy 数组
        data = select_data.to_numpy()
        #Add data to data_ist
        # 将数据添加到 data_list 中
        data_list.append(data)
    #Concatenate the data of all files into a large NumPy array
    # 将所有文件的数据拼接成一个大的 NumPy 数组
    data_array = np.array(data_list)  # shape: (site_size, seq_len, feature_size)
    data_array = data_array.transpose(1,0,2)# shape: (seq_len, site_size, feature_size)
    return data_array

In [4]:
# EarlyStopping，patience:allowing the maximum number of training epochs without improvement in validation loss; delta： When the validation loss improves delta at least compared to the optimal loss, it is considered to have improved
# 早停机制,patience验证损失未改善情况下允许训练的最大次数；delta：当验证损失比最佳损失至少改善delta则认为有所改善
class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [5]:
#Create a sliding window
#制作滑动窗口
def create_sequences(data, input_seq_len, output_seq_len, datay_site, target_col):
    X, y = [], []
    #Traverse all possible sequences
    # 遍历所有可能的序列
    for i in range(len(data) - input_seq_len - output_seq_len + 1):  # len(data) 是 seq_len
        #Create input data，shape:(site_size=9, input_seq_len=72, feature_size=9)
        # 制作输入序列数据，形状是 (site_size=9, input_seq_len=72, feature_size=9)
        X_sample = data[i:i + input_seq_len,:,  :]  # 选择 seq_len 范围的数据
        X.append(X_sample)
        
        #Create output data，shape:(target_site_size=1, output_seq_len, target_feature_size=1)
        # 制作输出序列数据，形状是 (target_site_size=1, output_seq_len, target_feature_size=1)
        target_data = data[i + input_seq_len:i + input_seq_len + output_seq_len, datay_site,  target_col]
        y.append(target_data.reshape(output_seq_len,1,  1))  
        
    return np.array(X), np.array(y)


In [6]:
# 定义LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size,dropout):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # 批量计算
        out = self.dropout(lstm_out[:, -1, :])
        out = self.fc(out)  # 仅使用最后时间步的隐藏状态#(batch_size,seq_len,hidden_size)
        #out = self.fc(lstm_out[:, :output_seq_len, :])
        #print(out.shape)
        return out

In [7]:
# 转换为Tensor
def to_tensor(arr):
    return torch.tensor(arr, dtype=torch.float32)

In [8]:
#评估指标
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

# 计算 RMSE、R2 和 MAE
def calculate_metrics(predictions, actuals):
    # 计算 RMSE
    rmse = np.sqrt(np.mean((predictions - actuals) ** 2))
    
    # 计算 R2
    r2 = r2_score(actuals, predictions)
    
    # 计算 MAE
    mae = mean_absolute_error(actuals, predictions)
    
    return rmse, r2, mae

## 定义函数（不同循环需修改）Define function (different loops need to be modified)

In [9]:
# LSTM训练和验证循环
def LSTM_train_and_evaluate(data, param_grid, datay_site, target_site_index, target_col, input_feature):              
    for params in ParameterGrid(param_grid):
        epoch_logs=[]
        input_seq_len = params['input_seq_len']  # 输入序列长度,Input sequence length
        output_seq_len = params['output_seq_len']
        batch_size = params['batch_size']
        learning_rate = params['learning_rate']
        LSTM_input_size = params['LSTM_input_size']
        LSTM_hidden_size = params['hidden_size']
        LSTM_num_layers = params['num_layers']
        dropout = params['dropout']
        
        print(f"Training with parameters: {params}")
        early_stopping = EarlyStopping(patience=10, delta=0.001)#早停
        best_model_state = None
        lstm_model = LSTMModel(LSTM_input_size, LSTM_hidden_size, LSTM_num_layers, output_seq_len,dropout)
        lstm_model.to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)       
        epochs=200
        
        #数据划分
        X, y = create_sequences(data, input_seq_len, output_seq_len, datay_site, target_col)
        y = y.squeeze((-1,-2))
        #Divide the training set，validation set and test set 7:1:2
        # 划分训练集验证集和测试集7：1：2
        train_size, val_size, test_size = 0.7, 0.1, 0.2
        X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=test_size/(train_size+val_size+test_size), shuffle=False)
        X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_size/(train_size+val_size), shuffle=False)
        X_train, X_val, X_test = to_tensor(X_train), to_tensor(X_val), to_tensor(X_test)
        y_train, y_val, y_test = to_tensor(y_train), to_tensor(y_val), to_tensor(y_test)
        # create DataLoader
        train_dataset = TensorDataset(X_train, y_train)
        val_dataset = TensorDataset(X_val, y_val)
        test_dataset = TensorDataset(X_test, y_test)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
        
        
        for epoch in range(epochs):
            start_time=time.time()
            lstm_model.train()
            train_loss = 0
            for X_origin, y_batch in train_loader:
                len_1 = len(input_feature) if isinstance(input_feature, (list, tuple, set)) else 1
                len_2 = len(target_site_index) if isinstance(target_site_index, (list, tuple, set)) else 1
                if len_1+len_2==2:
                    X_batch = X_origin[:,:,target_site_index,input_feature].unsqueeze(-1)
                if len_1+len_2==18:
                    X_batch = X_origin.view(X_origin.size(0),X_origin.size(1),-1)    
                else:
                    X_batch = X_origin[:,:,target_site_index,input_feature].squeeze()#(batch_size,seq_len, site_size, combined_feature_size)
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                #print(y_batch.shape)
                optimizer.zero_grad()
                outputs = lstm_model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            train_loss /= len(train_loader)
            
            # 验证
            lstm_model.eval()
            val_loss = 0
            with torch.no_grad():
                for X_origin, y_batch in val_loader:
                    len_1 = len(input_feature) if isinstance(input_feature, (list, tuple, set)) else 1
                    len_2 = len(target_site_index) if isinstance(target_site_index, (list, tuple, set)) else 1
                    if len_1+len_2==2:
                        X_batch = X_origin[:,:,target_site_index,input_feature].unsqueeze(-1)
                    if len_1+len_2==18:
                        X_batch = X_origin.view(X_origin.size(0),X_origin.size(1),-1)
                    else:
                        X_batch = X_origin[:,:,target_site_index,input_feature].squeeze()
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    outputs = lstm_model(X_batch)
                    loss = criterion(outputs, y_batch)
                    val_loss += loss.item()

            val_loss /= len(val_loader)
            end_time=time.time()
            epoch_duration=end_time-start_time
            epoch_logs.append([epoch+1,train_loss,val_loss,epoch_duration])
            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

            if val_loss < early_stopping.best_loss - early_stopping.delta:
                best_model_state = lstm_model.state_dict()  # 保存模型参数
            early_stopping(val_loss)
            if early_stopping.early_stop:
                print("Early stopping triggered.")
                break
        # 回退到最佳模型
        if best_model_state is not None:
            lstm_model.load_state_dict(best_model_state)
            print("Model restored to the best validation state.")


        # LSTM测试集评估
        lstm_model.eval()
        predictions, actuals = [], []
        with torch.no_grad():
            for X_origin, y_batch in test_loader:
                len_1 = len(input_feature) if isinstance(input_feature, (list, tuple, set)) else 1
                len_2 = len(target_site_index) if isinstance(target_site_index, (list, tuple, set)) else 1
                if len_1+len_2==2:
                    X_batch = X_origin[:,:,target_site_index,input_feature].unsqueeze(-1)
                if len_1+len_2==18:
                    X_batch = X_origin.view(X_origin.size(0),X_origin.size(1),-1)
                else:
                    X_batch = X_origin[:,:,target_site_index,input_feature].squeeze()
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = lstm_model(X_batch)
                predictions.append(outputs.cpu().numpy())  # 将数据移回CPU
                actuals.append(y_batch.cpu().numpy())      # 将数据移回CPU

        predictions = np.concatenate(predictions, axis=0)
        actuals = np.concatenate(actuals, axis=0)
        mse = mean_squared_error(actuals, predictions)
        print(f"Test MSE: {mse:.4f}")
    return predictions, actuals, lstm_model, test_loader, epoch_logs

#lstm model sensitivity
#LSTM特征重要性分析
def LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='noise'):
    """
    计算特征的重要性：对于每个特征，在其置零或加噪声时，观察对模型输出的影响。
    """
    feature_importances_all = []
    for X_origin, y_batch in test_loader:
        feature_importances = []
        len_1 = len(input_feature) if isinstance(input_feature, (list, tuple, set)) else 1
        len_2 = len(target_site_index) if isinstance(target_site_index, (list, tuple, set)) else 1
        if len_1+len_2==2:
            X_batch = X_origin[:,:,target_site_index,input_feature].unsqueeze(-1)
        if len_1+len_2==18:
            X_batch = X_origin.view(X_origin.size(0),X_origin.size(1),-1)
        else:
            X_batch = X_origin[:,:,target_site_index,input_feature].squeeze()
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        lstm_model.eval()
        baseline_output = lstm_model(X_batch).detach().cpu().numpy()  # 将数据移回CPU
        for j in range(X_batch.shape[2]):  # 遍历特征
            perturbed_output = []
            X_perturbed = X_batch.clone()
            if baseline == 'zero':
                X_perturbed[:, :, j] = 0  # 将第 i 个特征置零
            elif baseline == 'noise':
                noise = torch.randn_like(X_perturbed[:, :, j])
                X_perturbed[:, :, j] += noise*X_perturbed[:, :, j]  # 添加噪声
            outputs = lstm_model(X_perturbed)
            perturbed_output.append(outputs.detach().cpu().numpy())
            importance = np.mean(np.abs(perturbed_output - baseline_output))
            feature_importances.append(importance)
        feature_importances_all.append(feature_importances)
    return feature_importances_all

## 单站点单特征Single site single feature

In [28]:
#单站点单特征循环Single site single feature
#change the Define function.input: X_origin[:,:,target_site_index,input_feature].unsqueeze(-1),LSTM_input_size = 1
#超参数优化
param_grid = {
    'LSTM_input_size' : [1],
    'hidden_size': [64],#32,64,128
    'num_layers': [2],#1, 2, 3
    'learning_rate': [0.001],#0.001, 0.002, 0.005
    'batch_size': [16],#8，16, 32
    'dropout': [0.2],#0.1,0.2,0.3
    'input_seq_len' : [72],#24,48,72
    'output_seq_len' : [1]
}
best_accuracy = 0
best_params = {}

all_output_metrics = []
all_output_importance = []
folder_path = 'E:\LYQ\data'
#'E:\LYQ\data'
#'E:/your/data/folder_path'
data = load_data(folder_path)
for model_sites in range(9):
    output_metrics = []
    output_importance = []
    for model_input_feature in range(9):
        #Model hyperparameters
        datay_site = model_sites
        target_site_index = model_sites
        target_col = 2  # 目标特征（例如 特征0）,Target feature (e.g. feature 0)
        input_feature = model_input_feature#0:WT,1:pH,2:DO,3:EC,4:TSS,5:CODMN,6:AN,7:TP,8:TN

        target_site_index = model_sites  # 目标站点（例如 站点0）,Target site (e.g. site 0)
        predictions, actuals, lstm_model, test_loader, epoch_logs = LSTM_train_and_evaluate(data,param_grid,datay_site, target_site_index, target_col, input_feature)        #Calculate accuracy
        # 计算精度
        rmse, r2, mae = calculate_metrics(predictions, actuals)
        #result
        # 输出结果
        print(f"predictions shape: {predictions.shape}")
        print(f"actuals shape: {actuals.shape}")
        print(f"Test RMSE: {rmse:.4f}")
        print(f"Test R2: {r2:.4f}")
        print(f"Test MAE: {mae:.4f}")
        feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='noise')),axis= 0)
        print(feature_importances)
        
        output_metrics.append([rmse, r2, mae])
        output_importance.append(feature_importances)
        epoch_logs = pd.DataFrame(epoch_logs,columns=['Epoch','Train_Loss','Val_loss','Time'])
        epoch_logs.to_csv(f"LSTM_logs_1site_1feature_batch_site{model_sites}_feature{model_input_feature}.csv",index=False)
        
    all_output_metrics.append(pd.DataFrame(output_metrics,columns=['RMSE','R2','MAE']))
    all_output_importance.append(pd.DataFrame({f"N{model_sites}" : output_importance.values}, index = ['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']))
    torch.save(lstm_model,f"LSTM_N{target_site_index+1}_1Feature1Site_targetDO.pth")
    
result_output_metrics = pd.concat(all_output_metrics,axis = 1)
result_output_importance = pd.concat(all_output_importance,axis = 1)
result_output_metrics.index = result_output_importance.index = ['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']

result_output_metrics.to_csv(f"LSTM_1site_1feature_batch_metrics.csv",index=False)
result_output_importance.to_csv(f"LSTM_1site_1feature_batch_importance.csv",index=False)
['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']

Training with parameters: {'LSTM_input_size': 1, 'batch_size': 16, 'dropout': 0.2, 'hidden_size': 64, 'input_seq_len': 72, 'learning_rate': 0.001, 'num_layers': 2, 'output_seq_len': 1}
Epoch 1/200, Train Loss: 3.3434, Val Loss: 1.0462
Epoch 2/200, Train Loss: 1.8739, Val Loss: 1.1699
Epoch 3/200, Train Loss: 1.7709, Val Loss: 0.9568
Epoch 4/200, Train Loss: 1.7730, Val Loss: 1.0404
Epoch 5/200, Train Loss: 1.6822, Val Loss: 1.0330
Epoch 6/200, Train Loss: 1.6112, Val Loss: 0.9890
Epoch 7/200, Train Loss: 1.5718, Val Loss: 1.2011
Epoch 8/200, Train Loss: 1.5409, Val Loss: 0.7910
Epoch 9/200, Train Loss: 1.5354, Val Loss: 0.9658
Epoch 10/200, Train Loss: 1.5097, Val Loss: 0.9647
Epoch 11/200, Train Loss: 1.4921, Val Loss: 0.9482
Epoch 12/200, Train Loss: 1.4879, Val Loss: 0.7971
Epoch 13/200, Train Loss: 1.4774, Val Loss: 0.9863
Epoch 14/200, Train Loss: 1.4396, Val Loss: 0.6695
Epoch 15/200, Train Loss: 1.4237, Val Loss: 0.6673
Epoch 16/200, Train Loss: 1.4094, Val Loss: 0.7113
Epoch 17

## 单站点所有特征Single site all feature

In [11]:
#单站点所有特征循环,Single site all feature
#change the Define function input:X_origin[:,:,target_site_index,:].squeeze(),LSTM_input_size = 9
#超参数优化
param_grid = {
    'LSTM_input_size' : [9],
    'hidden_size': [64],#32,64,128
    'num_layers': [2],#1, 2, 3
    'learning_rate': [0.001],#0.001, 0.002, 0.005
    'batch_size': [16],#8，16, 32
    'dropout': [0.2],#0.1,0.2,0.3
    'input_seq_len' : [72],#24,48,72
    'output_seq_len' : [1]
}
best_accuracy = 0
best_params = {}

all_output_metrics = []
all_output_importance = []
folder_path = 'E:\LYQ\data'
#'E:\LYQ\data'
#'E:/your/data/folder_path'
data = load_data(folder_path)
print(data.shape)
target_col = 2  # 目标特征（例如 特征0）,Target feature (e.g. feature 0)
input_feature = [0,1,2,3,4,5,6,7,8]#0:WT,1:pH,2:DO,3:EC,4:TSS,5:CODMN,6:AN,7:TP,8:TN  


for model_sites in range(1):
    #Model hyperparameters
    # 模型超参数
    datay_site = model_sites
    target_site_index = model_sites
    predictions, actuals, lstm_model, test_loader, epoch_logs = LSTM_train_and_evaluate(data,param_grid, datay_site, target_site_index, target_col, input_feature)
    #Calculate accuracy
    # 计算精度
    rmse, r2, mae = calculate_metrics(predictions, actuals)
    #result
    # 输出结果
    print(f"predictions shape: {predictions.shape}")
    print(f"actuals shape: {actuals.shape}")
    print(f"Test RMSE: {rmse:.4f}")
    print(f"Test R2: {r2:.4f}")
    print(f"Test MAE: {mae:.4f}")
    feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='noise')),axis= 0)
    print(feature_importances)

    all_output_metrics.append([rmse, r2, mae])
    all_output_importance.append(pd.DataFrame({f"N{model_sites}" : feature_importances.values}, index = ['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']))
    epoch_logs = pd.DataFrame(epoch_logs,columns=['Epoch','Train_Loss','Val_loss','Time'])
    epoch_logs.to_csv(f"LSTM_logs_1siteallfeature_batch_site{model_sites}.csv",index=False)

    torch.save(lstm_model,f"LSTM_N{target_site_index+1}_AllFeature1Site_targetDO.pth")

result_output_metrics = pd.DataFrame(all_output_metrics,columns=['RMSE','R2','MAE'])
result_output_importance = pd.concat(all_output_importance,axis = 1)

result_output_metrics.to_csv(f"LSTM_1siteallfeature_batch_metrics_.csv",index=False)
result_output_importance.to_csv(f"LSTM_1siteallfeature_logs_one_batch_importance.csv",index=False)
#['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']

(30642, 9, 9)
Training with parameters: {'LSTM_input_size': 9, 'batch_size': 16, 'dropout': 0.2, 'hidden_size': 64, 'input_seq_len': 72, 'learning_rate': 0.001, 'num_layers': 2, 'output_seq_len': 1}
Epoch 1/200, Train Loss: 2.2908, Val Loss: 0.5853
Epoch 2/200, Train Loss: 0.4792, Val Loss: 0.4542
Epoch 3/200, Train Loss: 0.4410, Val Loss: 0.5107
Epoch 4/200, Train Loss: 0.4208, Val Loss: 0.5473
Epoch 5/200, Train Loss: 0.3853, Val Loss: 0.4644
Epoch 6/200, Train Loss: 0.3833, Val Loss: 0.4832
Epoch 7/200, Train Loss: 0.3748, Val Loss: 0.4648
Epoch 8/200, Train Loss: 0.3544, Val Loss: 0.3354
Epoch 9/200, Train Loss: 0.3495, Val Loss: 0.5461
Epoch 10/200, Train Loss: 0.3327, Val Loss: 0.3919
Epoch 11/200, Train Loss: 0.3239, Val Loss: 0.3242
Epoch 12/200, Train Loss: 0.3023, Val Loss: 0.3547
Epoch 13/200, Train Loss: 0.2974, Val Loss: 0.3857
Epoch 14/200, Train Loss: 0.2857, Val Loss: 0.3866
Epoch 15/200, Train Loss: 0.2758, Val Loss: 0.3286
Epoch 16/200, Train Loss: 0.2764, Val Loss: 0

## 所有站点单特征all site single feature

In [12]:
#所有站点单特征循环all site single feature
#change the Define function input:X_origin[:,:,:,input_feature].squeeze(),LSTM_input_size = 9
#超参数优化
param_grid = {
    'LSTM_input_size' : [9],
    'hidden_size': [64],#32,64,128
    'num_layers': [2],#1, 2, 3
    'learning_rate': [0.001],#0.001, 0.002, 0.005
    'batch_size': [16],#8，16, 32
    'dropout': [0.2],#0.1,0.2,0.3
    'input_seq_len' : [72],#24,48,72
    'output_seq_len' : [1]
}
best_accuracy = 0
best_params = {}

all_output_metrics = []
all_output_importance = []
folder_path = 'E:\LYQ\data'
#'E:\LYQ\data'
#'E:/your/data/folder_path'
data = load_data(folder_path)
print(data.shape)
target_col = 2  # 目标特征（例如 特征0）,Target feature (e.g. feature 0)
target_site_index = [0,1,2,3,4,5,6,7,8]#range(9)
all_output_metrics = []
all_output_importance = []

for model_sites in range(9):
    output_metrics = []
    output_importance = []
    for model_input_feature in range(9):
        #Model hyperparameters
        # 模型超参数
        datay_site = model_sites

        input_feature = model_input_feature#0:WT,1:pH,2:DO,3:EC,4:TSS,5:CODMN,6:AN,7:TP,8:TN

        predictions, actuals, lstm_model, test_loader, epoch_logs = LSTM_train_and_evaluate(data,param_grid, datay_site, target_site_index, target_col, input_feature)
        #Calculate accuracy
        # 计算精度
        rmse, r2, mae = calculate_metrics(predictions, actuals)
        #result
        # 输出结果
        print(f"predictions shape: {predictions.shape}")
        print(f"actuals shape: {actuals.shape}")
        print(f"Test RMSE: {rmse:.4f}")
        print(f"Test R2: {r2:.4f}")
        print(f"Test MAE: {mae:.4f}")
        feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='zero')),axis= 0)
        print(feature_importances)
        
        feature_importanes= pd.DataFrame({f"F{model_input_feature}" : feature_importances.values}, index = [f'N{s}' for s in range(9)])
        
        output_metrics.append([rmse, r2, mae])
        output_importance.append(feature_importances)
        epoch_logs = pd.DataFrame(epoch_logs,columns=['Epoch','Train_Loss','Val_loss','Time'])
        epoch_logs.to_csv(f"LSTM_logs_allsite_1feature_batch_site{model_sites}_feature{model_input_feature}.csv",index=False)  
        
    all_output_metrics.append(pd.DataFrame(output_metrics,columns=['RMSE','R2','MAE'], index = ['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']))
    all_output_importance = pd.concat(output_importance,axis = 1)
    all_output_importance.to_csv(f"LSTM_al1site_1feature_batch_importance_site{model_sites}.csv",index=False)

    torch.save(lstm_model,f"LSTM_N{target_site_index+1}_1FeatureAllSite_targetDO.pth")
    
result_output_metrics = pd.concat(all_output_metrics,axis = 1)

result_output_metrics.to_csv(f"LSTM_al1site_1feature_batch_metrics.csv",index=False)

# all_output_metrics.to_csv(f"LSTM_allsite_1feature_batch_metrics备用查看数据.csv",index=False)
#['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']

(30642, 9, 9)
Training with parameters: {'LSTM_input_size': 9, 'batch_size': 16, 'dropout': 0.2, 'hidden_size': 64, 'input_seq_len': 72, 'learning_rate': 0.001, 'num_layers': 2, 'output_seq_len': 1}
Epoch 1/200, Train Loss: 3.3912, Val Loss: 0.9712
Epoch 2/200, Train Loss: 1.7313, Val Loss: 0.9760
Epoch 3/200, Train Loss: 1.6899, Val Loss: 1.0574
Epoch 4/200, Train Loss: 1.6682, Val Loss: 0.9305
Epoch 5/200, Train Loss: 1.5761, Val Loss: 1.0228
Epoch 6/200, Train Loss: 1.4998, Val Loss: 0.9627
Epoch 7/200, Train Loss: 1.3818, Val Loss: 0.9327
Epoch 8/200, Train Loss: 1.3337, Val Loss: 0.7837
Epoch 9/200, Train Loss: 1.2906, Val Loss: 0.9181
Epoch 10/200, Train Loss: 1.2782, Val Loss: 0.8577
Epoch 11/200, Train Loss: 1.2528, Val Loss: 0.9729
Epoch 12/200, Train Loss: 1.2150, Val Loss: 1.4816
Epoch 13/200, Train Loss: 1.1874, Val Loss: 0.8125
Epoch 14/200, Train Loss: 1.2008, Val Loss: 0.9700
Epoch 15/200, Train Loss: 1.1664, Val Loss: 0.8087
Epoch 16/200, Train Loss: 1.1652, Val Loss: 0

In [10]:

#所有站点单特征循环all site single feature
#change the Define function input:X_origin[:,:,:,input_feature].squeeze(),LSTM_input_size = 9
#超参数优化
param_grid = {
    'LSTM_input_size' : [8],
    'hidden_size': [64],#32,64,128
    'num_layers': [3],#1, 2, 3
    'learning_rate': [0.001],#0.001, 0.002, 0.005
    'batch_size': [16],#8，16, 32
    'dropout': [0.3],#0.1,0.2,0.3
    'input_seq_len' : [72],#24,48,72
    'output_seq_len' : [1]
}
best_accuracy = 0
best_params = {}

all_output_metrics = []
all_output_importance = []
folder_path = 'E:\LYQ\data'
#'E:\LYQ\data'
#'E:/your/data/folder_path'
data = load_data(folder_path)
print(data.shape)
target_col = 2  # 目标特征（例如 特征0）,Target feature (e.g. feature 0)
range9 = list(range(9))
all_output_metrics = []
all_output_importance = []

for model_sites in range(9):
    output_metrics = []
    output_importance = []
    for model_input_feature in range(1):
        #Model hyperparameters
        # 模型超参数
        datay_site = model_sites
        target_site_index = range9[:model_sites]+range9[model_sites+1:]
        input_feature = 2#0:WT,1:pH,2:DO,3:EC,4:TSS,5:CODMN,6:AN,7:TP,8:TN

        predictions, actuals, lstm_model, test_loader, epoch_logs = LSTM_train_and_evaluate(data,param_grid, datay_site, target_site_index, target_col, input_feature)
        #Calculate accuracy
        # 计算精度
        rmse, r2, mae = calculate_metrics(predictions, actuals)
        #result
        # 输出结果
        print(f"predictions shape: {predictions.shape}")
        print(f"actuals shape: {actuals.shape}")
        print(f"Test RMSE: {rmse:.4f}")
        print(f"Test R2: {r2:.4f}")
        print(f"Test MAE: {mae:.4f}")
        feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='zero')),axis= 0)
        print(feature_importances)
        
#         feature_importanes= pd.DataFrame({f"F{model_input_feature}" : feature_importances.values}, index = [f'N{s}' for s in range(9)])
        
#         output_metrics.append([rmse, r2, mae])
#         output_importance.append(feature_importances)
#         epoch_logs = pd.DataFrame(epoch_logs,columns=['Epoch','Train_Loss','Val_loss','Time'])
#         epoch_logs.to_csv(f"LSTM_logs_allsite_1feature_batch_site{model_sites}_feature{model_input_feature}.csv",index=False)  
        
#     all_output_metrics.append(pd.DataFrame(output_metrics,columns=['RMSE','R2','MAE'], index = ['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']))
#     all_output_importance = pd.concat(output_importance,axis = 1)
#     all_output_importance.to_csv(f"LSTM_al1site_1feature_batch_importance_site{model_sites}.csv",index=False)

#     torch.save(lstm_model,f"LSTM_N{target_site_index+1}_1FeatureAllSite_targetDO.pth")
    
# result_output_metrics = pd.concat(all_output_metrics,axis = 1)

# result_output_metrics.to_csv(f"LSTM_al1site_1feature_batch_metrics.csv",index=False)

# all_output_metrics.to_csv(f"LSTM_allsite_1feature_batch_metrics备用查看数据.csv",index=False)
#['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']

(30642, 9, 9)
Training with parameters: {'LSTM_input_size': 8, 'batch_size': 16, 'dropout': 0.2, 'hidden_size': 64, 'input_seq_len': 72, 'learning_rate': 0.001, 'num_layers': 2, 'output_seq_len': 1}
Epoch 1/200, Train Loss: 3.3546, Val Loss: 0.9230
Epoch 2/200, Train Loss: 1.2254, Val Loss: 0.9439
Epoch 3/200, Train Loss: 1.1671, Val Loss: 1.1216
Epoch 4/200, Train Loss: 1.0972, Val Loss: 1.2287
Epoch 5/200, Train Loss: 1.0293, Val Loss: 1.4855
Epoch 6/200, Train Loss: 0.9620, Val Loss: 1.6070
Epoch 7/200, Train Loss: 0.8755, Val Loss: 1.8171
Epoch 8/200, Train Loss: 0.8064, Val Loss: 1.7334
Epoch 9/200, Train Loss: 0.7363, Val Loss: 1.2975
Epoch 10/200, Train Loss: 0.7073, Val Loss: 1.5761
Epoch 11/200, Train Loss: 0.6640, Val Loss: 1.3089
Early stopping triggered.
Model restored to the best validation state.
Test MSE: 1.0147
predictions shape: (6114, 1)
actuals shape: (6114, 1)
Test RMSE: 1.0073
Test R2: 0.5781
Test MAE: 0.7357
0    1.554133
1    1.105961
2    0.618451
3    1.272881


Epoch 2/200, Train Loss: 1.1266, Val Loss: 0.8536
Epoch 3/200, Train Loss: 1.0810, Val Loss: 1.0222
Epoch 4/200, Train Loss: 1.0385, Val Loss: 0.8473
Epoch 5/200, Train Loss: 0.9832, Val Loss: 1.1354
Epoch 6/200, Train Loss: 0.9364, Val Loss: 1.2395
Epoch 7/200, Train Loss: 0.9165, Val Loss: 1.3763
Epoch 8/200, Train Loss: 0.8620, Val Loss: 0.8860
Epoch 9/200, Train Loss: 0.8338, Val Loss: 1.5593
Epoch 10/200, Train Loss: 0.8266, Val Loss: 1.1515
Epoch 11/200, Train Loss: 0.7663, Val Loss: 1.5017
Epoch 12/200, Train Loss: 0.7373, Val Loss: 1.4058
Epoch 13/200, Train Loss: 0.7127, Val Loss: 1.5514
Epoch 14/200, Train Loss: 0.7029, Val Loss: 1.4588
Early stopping triggered.
Model restored to the best validation state.
Test MSE: 1.3703
predictions shape: (6114, 1)
actuals shape: (6114, 1)
Test RMSE: 1.1706
Test R2: 0.4421
Test MAE: 1.0155
0    0.566114
1    0.845455
2    1.431105
3    0.876834
4    1.439783
5    0.666207
6    2.039798
7    1.068767
dtype: float32
Training with parameters:

## 所有站点所有特征all site all feature

In [ ]:
#所有站点所有特征循环all site all feature 
#change the Define function input:X_origin[:,:,:,:].squeeze(),X_origin = X_origin.reshape(16,72,81),LSTM_input_size = 81
#超参数优化
param_grid = {
    'LSTM_input_size' : [81],
    'hidden_size': [64],#32,64,128
    'num_layers': [2],#1, 2, 3
    'learning_rate': [0.001],#0.001, 0.002, 0.005
    'batch_size': [16],#8，16, 32
    'dropout': [0.2],#0.1,0.2,0.3
    'input_seq_len' : [72],#24,48,72
    'output_seq_len' : [1]
}
best_accuracy = 0
best_params = {}

all_output_metrics = []
all_output_importance = []
folder_path = 'E:\LYQ\data'
#'E:\LYQ\data'
#'E:/your/data/folder_path'
data = load_data(folder_path)
print(data.shape)
target_col = 2  # 目标特征（例如 特征0）,Target feature (e.g. feature 0)
input_feature = [0,1,2,3,4,5,6,7,8]#0:WT,1:pH,2:DO,3:EC,4:TSS,5:CODMN,6:AN,7:TP,8:TN  
target_site_index = [0,1,2,3,4,5,6,7,8]#range(9)

all_output_metrics = []
all_output_importance = []

for model_sites in range(9):
    # 模型超参数

    predictions, actuals, lstm_model, test_loader, epoch_logs = LSTM_train_and_evaluate(data,param_grid, datay_site, target_site_index, target_col, input_feature)
    #Calculate accuracy
    # 计算精度
    rmse, r2, mae = calculate_metrics(predictions, actuals)
    #result
    # 输出结果
    print(f"predictions shape: {predictions.shape}")
    print(f"actuals shape: {actuals.shape}")
    print(f"Test RMSE: {rmse:.4f}")
    print(f"Test R2: {r2:.4f}")
    print(f"Test MAE: {mae:.4f}")
    feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='zero')),axis= 0)
    print(feature_importances)

    all_output_metrics.append([rmse, r2, mae])
    all_output_importance.append(pd.DataFrame({f"N{model_sites}" : feature_importances.values}, index=[f'F{s}' for s in range(81)]))
    epoch_logs = pd.DataFrame(epoch_logs,columns=['Epoch','Train_Loss','Val_loss','Time'])
    epoch_logs.to_csv(f"LSTM_logs_allsiteallfeature_batch_site{model_sites}.csv",index=False)
    torch.save(lstm_model,f"LSTM_N{target_site_index+1}_AllFeatureAllSite_targetDO.pth")
    
result_output_metrics = pd.DataFrame(output_metrics,columns=['RMSE','R2','MAE'])
result_output_importance = pd.concat(all_output_importance,axis = 1)

result_output_metrics.to_csv(f"LSTM_allsiteallfeature_batch_metrics_.csv",index=False)
result_output_importance.to_csv(f"LSTM_1siteallfeature_logs_one_batch_importance.csv",index=False)
# all_output_importance.to_csv(f"LSTM_allsiteallfeature_logs_one_batch_importance备用查看数据.csv",index=False)
#['WT','pH','DO','EC','TSS','CODMn','AN','TP','TN']

(30642, 9, 9)
Training with parameters: {'LSTM_input_size': 81, 'batch_size': 16, 'dropout': 0.2, 'hidden_size': 64, 'input_seq_len': 72, 'learning_rate': 0.001, 'num_layers': 2, 'output_seq_len': 1}
Epoch 1/200, Train Loss: 3.5644, Val Loss: 2.8110
Epoch 2/200, Train Loss: 1.7467, Val Loss: 2.9871
Epoch 3/200, Train Loss: 1.5520, Val Loss: 2.5800
Epoch 4/200, Train Loss: 1.4092, Val Loss: 2.9207
Epoch 5/200, Train Loss: 1.3192, Val Loss: 2.3488
Epoch 6/200, Train Loss: 1.2103, Val Loss: 2.7990
Epoch 7/200, Train Loss: 1.1642, Val Loss: 2.0542
Epoch 8/200, Train Loss: 1.1450, Val Loss: 2.2381
Epoch 9/200, Train Loss: 1.0917, Val Loss: 1.8651
Epoch 10/200, Train Loss: 1.1387, Val Loss: 1.9762
Epoch 11/200, Train Loss: 1.0953, Val Loss: 1.8019
Epoch 12/200, Train Loss: 1.0089, Val Loss: 1.8754
Epoch 13/200, Train Loss: 0.9826, Val Loss: 1.8519
Epoch 14/200, Train Loss: 0.9692, Val Loss: 1.7705
Epoch 15/200, Train Loss: 0.9370, Val Loss: 1.4801
Epoch 16/200, Train Loss: 0.8911, Val Loss: 

## 低氧情境

In [17]:
#所有站点单特征循环all site single feature
#change the Define function input:X_origin[:,:,:,input_feature].squeeze(),LSTM_input_size = 9
#超参数优化
param_grid = {
    'LSTM_input_size' : [9],
    'hidden_size': [64],#32,64,128
    'num_layers': [2],#1, 2, 3
    'learning_rate': [0.001],#0.001, 0.002, 0.005
    'batch_size': [16],#8，16, 32
    'dropout': [0.2],#0.1,0.2,0.3
    'input_seq_len' : [72],#24,48,72
    'output_seq_len' : [1]
}
best_accuracy = 0
best_params = {}
feature_importances_all = []
feature_importances_lowDO = []
folder_path = 'E:\LYQ\data'
#'E:\LYQ\data'
#'E:/your/data/folder_path'
data = load_data(folder_path)
print(data.shape)
target_col = 2  # 目标特征（例如 特征0）,Target feature (e.g. feature 0)
target_site_index = [0,1,2,3,4,5,6,7,8]#range(9)

for model_sites in range(9):
    #Model hyperparameters
    # 模型超参数
    datay_site = model_sites

    input_feature = 2#0:WT,1:pH,2:DO,3:EC,4:TSS,5:CODMN,6:AN,7:TP,8:TN

    predictions, actuals, lstm_model, test_loader, epoch_logs = LSTM_train_and_evaluate(data,param_grid, datay_site, target_site_index, target_col, input_feature)
    #Calculate accuracy
    # 计算精度
    rmse, r2, mae = calculate_metrics(predictions, actuals)
    #result
    # 输出结果
    print(f"predictions shape: {predictions.shape}")
    print(f"actuals shape: {actuals.shape}")
    print(f"Test RMSE: {rmse:.4f}")
    print(f"Test R2: {r2:.4f}")
    print(f"Test MAE: {mae:.4f}")
    feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='noise')),axis= 0)
    print(f"N{model_sites+1}总特征重要性：")
    print(feature_importances)   
    
    #数据划分
    X, y = create_sequences(data, 72, 1, datay_site, target_col)
    y = y.squeeze((-1,-2))
        
    mask = y.squeeze()<2
    X_filtered = X[mask]
    y_filtered = y[mask]
    X_filtered,y_filtered = to_tensor(X_filtered),to_tensor(y_filtered)
    print(X_filtered.shape)
    if X_filtered.shape[0]<5:
        continue
    dataset_filtered = TensorDataset(X_filtered, y_filtered)
    loader_filtered = DataLoader(dataset_filtered, batch_size=16, shuffle=True)
    
    feature_importances = np.mean(pd.DataFrame(LSTM_feature_importance_analysis(lstm_model, test_loader, target_site_index, input_feature, baseline='noise')),axis= 0)
    print(f"N{model_sites+1}低氧情境特征重要性：")
    print(feature_importances)
    feature_importances_lowDO.append(pd.DataFrame(feature_importances.values))
    
feature_importances_all = pd.concat(feature_importances_all,axis = 1)
feature_importances_lowDO = pd.concat(feature_importances_lowDO,axis = 1)
feature_importances_all.to_csv(f"feature_importances_all.csv",index=False)   
feature_importances_lowDO.to_csv(f"feature_importances_lowDO.csv",index=False)  


(30642, 9, 9)
Training with parameters: {'LSTM_input_size': 9, 'batch_size': 16, 'dropout': 0.2, 'hidden_size': 64, 'input_seq_len': 72, 'learning_rate': 0.001, 'num_layers': 2, 'output_seq_len': 1}
Epoch 1/200, Train Loss: 2.8349, Val Loss: 0.2447
Epoch 2/200, Train Loss: 0.4510, Val Loss: 0.0691
Epoch 3/200, Train Loss: 0.4116, Val Loss: 0.0978
Epoch 4/200, Train Loss: 0.3905, Val Loss: 0.1160
Epoch 5/200, Train Loss: 0.3871, Val Loss: 0.1017
Epoch 6/200, Train Loss: 0.3666, Val Loss: 0.0684
Epoch 7/200, Train Loss: 0.3531, Val Loss: 0.0970
Epoch 8/200, Train Loss: 0.3349, Val Loss: 0.0639
Epoch 9/200, Train Loss: 0.3214, Val Loss: 0.1410
Epoch 10/200, Train Loss: 0.3130, Val Loss: 0.0686
Epoch 11/200, Train Loss: 0.3022, Val Loss: 0.0664
Epoch 12/200, Train Loss: 0.2891, Val Loss: 0.0808
Epoch 13/200, Train Loss: 0.2803, Val Loss: 0.1091
Epoch 14/200, Train Loss: 0.2735, Val Loss: 0.0813
Epoch 15/200, Train Loss: 0.2612, Val Loss: 0.0608
Epoch 16/200, Train Loss: 0.2455, Val Loss: 0

Epoch 1/200, Train Loss: 1.0546, Val Loss: 0.1319
Epoch 2/200, Train Loss: 0.2937, Val Loss: 0.1463
Epoch 3/200, Train Loss: 0.2647, Val Loss: 0.1496
Epoch 4/200, Train Loss: 0.2338, Val Loss: 0.0973
Epoch 5/200, Train Loss: 0.2156, Val Loss: 0.1100
Epoch 6/200, Train Loss: 0.2037, Val Loss: 0.0981
Epoch 7/200, Train Loss: 0.1973, Val Loss: 0.0971
Epoch 8/200, Train Loss: 0.1907, Val Loss: 0.0925
Epoch 9/200, Train Loss: 0.1821, Val Loss: 0.1020
Epoch 10/200, Train Loss: 0.1767, Val Loss: 0.1131
Epoch 11/200, Train Loss: 0.1675, Val Loss: 0.1238
Epoch 12/200, Train Loss: 0.1661, Val Loss: 0.1019
Epoch 13/200, Train Loss: 0.1615, Val Loss: 0.1144
Epoch 14/200, Train Loss: 0.1569, Val Loss: 0.1237
Epoch 15/200, Train Loss: 0.1516, Val Loss: 0.1297
Epoch 16/200, Train Loss: 0.1486, Val Loss: 0.0881
Epoch 17/200, Train Loss: 0.1452, Val Loss: 0.0988
Epoch 18/200, Train Loss: 0.1407, Val Loss: 0.1016
Epoch 19/200, Train Loss: 0.1386, Val Loss: 0.0989
Epoch 20/200, Train Loss: 0.1346, Val Lo

Epoch 3/200, Train Loss: 0.2367, Val Loss: 0.0953
Epoch 4/200, Train Loss: 0.2136, Val Loss: 0.1408
Epoch 5/200, Train Loss: 0.2132, Val Loss: 0.1026
Epoch 6/200, Train Loss: 0.2014, Val Loss: 0.0981
Epoch 7/200, Train Loss: 0.1966, Val Loss: 0.0983
Epoch 8/200, Train Loss: 0.1883, Val Loss: 0.0979
Epoch 9/200, Train Loss: 0.1818, Val Loss: 0.0953
Epoch 10/200, Train Loss: 0.1775, Val Loss: 0.1006
Epoch 11/200, Train Loss: 0.1712, Val Loss: 0.0895
Epoch 12/200, Train Loss: 0.1648, Val Loss: 0.0927
Epoch 13/200, Train Loss: 0.1619, Val Loss: 0.0889
Epoch 14/200, Train Loss: 0.1573, Val Loss: 0.0944
Epoch 15/200, Train Loss: 0.1555, Val Loss: 0.0950
Epoch 16/200, Train Loss: 0.1482, Val Loss: 0.0986
Epoch 17/200, Train Loss: 0.1454, Val Loss: 0.0966
Epoch 18/200, Train Loss: 0.1449, Val Loss: 0.0886
Epoch 19/200, Train Loss: 0.1402, Val Loss: 0.0896
Epoch 20/200, Train Loss: 0.1378, Val Loss: 0.0970
Epoch 21/200, Train Loss: 0.1368, Val Loss: 0.0937
Early stopping triggered.
Model restor

ValueError: No objects to concatenate

## 其他other

In [ ]:
data_iter = iter(test_loader)  # 创建一个迭代器
X_sample, y_sample = next(data_iter)  # 获取一个批次数据
print(X_sample.shape)